## Controlling webcam

In [1]:
import numpy as np
import cv2 as cv
from PIL import Image
import ipywidgets
from IPython.display import clear_output
import os
from io import BytesIO

In [2]:
WIDTH = 1280
HEIGHT = 1024
SIZE = (WIDTH, HEIGHT)

In [3]:


INTERFACE = "usb" # usb, csi

def get_source(id, interface=INTERFACE):
    if id == "left":
        id = 0
    elif id == "right":
        id = 1

    if interface == "usb":
        return id
    elif interface == "csi":
        return (
            "nvarguscamerasrc sensor-id=%d sensor-mode=%d ! "
            "video/x-raw(memory:NVMM), "
            "width=(int)%d, height=(int)%d, "
            "format=(string)NV12, framerate=(fraction)%d/1 ! "
            "nvvidconv flip-method=%d ! "
            "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
            "videoconvert ! "
            "video/x-raw, format=(string)BGR ! appsink"
            % (
                id, # camera id
                3, # mode
                640, # width
                480, # height
                10, # frame rate
                0, # flip method
                640, # display width
                480, # display height
            )
        )
    else:
        raise Exception("Unknown camera source.")

def camera_capture():
    left = cv.VideoCapture(get_source("left"))
    left.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
    left.set(cv.CAP_PROP_FRAME_HEIGHT, 1024)

    right = cv.VideoCapture(get_source("right"))
    right.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
    right.set(cv.CAP_PROP_FRAME_HEIGHT, 1024)

    left.grab()
    right.grab()

    _, left_frame = left.retrieve()
    _, right_frame = right.retrieve()

    left_frame = cv.cvtColor(left_frame, cv.COLOR_BGR2GRAY)
    right_frame = cv.cvtColor(right_frame, cv.COLOR_BGR2GRAY)

    left.release()
    right.release()
    return np.hstack((left_frame, right_frame))

## Calibration

### Collecting images (slow with vscode notebooks, works well with jupyter)

In [6]:
import threading
import os

running = True

def save(*args):
    count = len(os.listdir("calibration/left"))
    with open(f"calibration/left/{count}.jpg", "wb") as f:
        f.write(camera_widget_left.value)
    with open(f"calibration/right/{count}.jpg", "wb") as f:
        f.write(camera_widget_right.value)

camera_widget_left = ipywidgets.Image(width=400)
camera_widget_right = ipywidgets.Image(width=400)
save_button = ipywidgets.Button(description="Save")
save_button.on_click(save)
all_widgets = ipywidgets.VBox((
    save_button,
    ipywidgets.HBox((camera_widget_left, camera_widget_right)),
))

def live_execution():
    global camera_widget
    display(all_widgets)

    global running
    camera0 = cv.VideoCapture(get_source(0))
    camera0.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
    camera0.set(cv.CAP_PROP_FRAME_HEIGHT, 1024)

    camera1 = cv.VideoCapture(get_source(1))
    camera1.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
    camera1.set(cv.CAP_PROP_FRAME_HEIGHT, 1024)
    try: 
        while running:
            camera0.grab()
            camera1.grab()

            _, frame0 = camera0.retrieve()
            _, frame1 = camera1.retrieve()

            frame0 = cv.cvtColor(frame0, cv.COLOR_BGR2RGB)
            frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2RGB)
            
            stream0, stream1 = BytesIO(), BytesIO()
            Image.fromarray(np.uint8(frame0)).save(stream0, format="jpeg")
            Image.fromarray(np.uint8(frame1)).save(stream1, format="jpeg")
            camera_widget_left.value = stream0.getvalue()
            camera_widget_right.value = stream1.getvalue()
    except KeyboardInterrupt:
        pass
    print("Stream Stopped")
    camera0.release()
    camera1.release()

In [7]:
# thread = threading.Thread(target=live_execution)
# thread.start()

In [8]:
running = False

### Loading images

In [9]:
left_images = []
right_images = []
left_images_pair = []
right_images_pair = []

for f in os.listdir("calibration/left"):
    image = cv.imread("calibration/left/" + f)
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    left_images.append(image)

for f in os.listdir("calibration/right"):
    image = cv.imread("calibration/right/" + f)
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    right_images.append(image)

for f in os.listdir("calibration/left-right/left/"):
    image = cv.imread("calibration/left-right/left/" + f)
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    left_images_pair.append(image)

for f in os.listdir("calibration/left-right/right/"):
    image = cv.imread("calibration/left-right/right/" + f)
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    right_images_pair.append(image)

if len(left_images_pair) != len(right_images_pair):
    print("ERROR Pairs incomplete.")

print(len(left_images), "+", len(left_images_pair), "Left images.")
print(len(right_images), "+", len(right_images_pair), "Right images.")
print(len(left_images_pair), "Left-Right pairs")

59 + 69 Left images.
58 + 69 Right images.
69 Left-Right pairs


### Finding chessboard corners

In [10]:
CHESSBOARD_SQUARE_SIZE = 0.02384 # m
CHESSBOARD_SIZE = (7, 7)

In [9]:
CRITERIA = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

left_obj_points = []
right_obj_points = []
left_image_points = []
right_image_points = []

obj_pair_points = []
left_image_pair_points = []
right_image_pair_points = []

for idx, (left_image, right_image) in enumerate(zip(left_images_pair, right_images_pair)):
    print("Processing pair", idx, end=": ")
    
    # Find chessboard corners
    ret_left, corners_left = cv.findChessboardCorners(left_image, (7, 7), cv.CALIB_CB_ADAPTIVE_THRESH | cv.CALIB_CB_NORMALIZE_IMAGE | cv.CALIB_CB_FAST_CHECK)
    ret_right, corners_right = cv.findChessboardCorners(right_image, (7, 7), cv.CALIB_CB_ADAPTIVE_THRESH | cv.CALIB_CB_NORMALIZE_IMAGE | cv.CALIB_CB_FAST_CHECK)

    # If no corners found, continue
    if not (ret_left and ret_right):
        print("ERROR")
        continue

    # Increase the accuracy of corner points
    corners_left = cv.cornerSubPix(left_image, corners_left, (11, 11), (-1, -1), CRITERIA)
    corners_right = cv.cornerSubPix(right_image, corners_right, (11, 11), (-1, -1), CRITERIA)
    
    # Save chessboard with drawn corners
    # left_image_corners = left_image.copy()
    # left_image_corners = cv.cvtColor(left_image_corners, cv.COLOR_GRAY2BGR)
    # cv.drawChessboardCorners(left_image_corners, (7, 7), corners_left, True)
    # cv.imwrite("calibration-corners/left/" + str(idx) + ".png", left_image_corners)
    # right_image_corners = right_image.copy()
    # right_image_corners = cv.cvtColor(right_image_corners, cv.COLOR_GRAY2BGR)
    # cv.drawChessboardCorners(right_image_corners, (7, 7), corners_right, True)
    # cv.imwrite("calibration-corners/right/" + str(idx) + ".png", right_image_corners)

    # Add chessboard points as object points
    obj = np.zeros((7 * 7, 3), np.float32)
    obj[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2) * CHESSBOARD_SQUARE_SIZE
    obj_pair_points.append(obj)
    left_obj_points.append(obj)
    right_obj_points.append(obj)

    # Add chessboard points as image points
    left_image_pair_points.append(corners_left)
    right_image_pair_points.append(corners_right)
    left_image_points.append(corners_left)
    right_image_points.append(corners_right)
     
    print("OK")

for idx, left_image in enumerate(left_images):
    print("Processing left image", idx, end=": ")
    
    # Find chessboard corners
    ret_left, corners_left = cv.findChessboardCorners(left_image, (7, 7), cv.CALIB_CB_ADAPTIVE_THRESH | cv.CALIB_CB_NORMALIZE_IMAGE | cv.CALIB_CB_FAST_CHECK)

    # If no corners found, continue
    if not ret_left:
        print("ERROR")
        continue

    # Increase the accuracy of corner points
    corners_left = cv.cornerSubPix(left_image, corners_left, (11, 11), (-1, -1), CRITERIA)

    # Add chessboard points as object points
    obj = np.zeros((7 * 7, 3), np.float32)
    obj[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2) * CHESSBOARD_SQUARE_SIZE
    left_obj_points.append(obj)

    # Add chessboard points as image points
    left_image_points.append(corners_left)
    print("OK")

for idx, right_image in enumerate(right_images):
    print("Processing right image", idx, end=": ")
    
    # Find chessboard corners
    ret_right, corners_right = cv.findChessboardCorners(right_image, (7, 7), cv.CALIB_CB_ADAPTIVE_THRESH | cv.CALIB_CB_NORMALIZE_IMAGE | cv.CALIB_CB_FAST_CHECK)

    # If no corners found, continue
    if not ret_right:
        print("ERROR")
        continue

    # Increase the accuracy of corner points
    corners_right = cv.cornerSubPix(right_image, corners_right, (11, 11), (-1, -1), CRITERIA)

    # Add chessboard points as object points
    obj = np.zeros((7 * 7, 3), np.float32)
    obj[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2) * CHESSBOARD_SQUARE_SIZE
    right_obj_points.append(obj)

    # Add chessboard points as image points
    right_image_points.append(corners_left)
    print("OK")

Processing pair 0: OK
Processing pair 1: OK
Processing pair 2: OK
Processing pair 3: OK
Processing pair 4: OK
Processing pair 5: OK
Processing pair 6: OK
Processing pair 7: OK
Processing pair 8: OK
Processing pair 9: OK
Processing pair 10: OK
Processing pair 11: OK
Processing pair 12: OK
Processing pair 13: OK
Processing pair 14: OK
Processing pair 15: OK
Processing pair 16: OK
Processing pair 17: OK
Processing pair 18: OK
Processing pair 19: OK
Processing pair 20: OK
Processing pair 21: OK
Processing pair 22: OK
Processing pair 23: OK
Processing pair 24: OK
Processing pair 25: OK
Processing pair 26: OK
Processing pair 27: OK
Processing pair 28: OK
Processing pair 29: OK
Processing pair 30: OK
Processing pair 31: OK
Processing pair 32: ERROR
Processing pair 33: ERROR
Processing pair 34: OK
Processing pair 35: OK
Processing pair 36: OK
Processing pair 37: ERROR
Processing pair 38: OK
Processing pair 39: OK
Processing pair 40: OK
Processing pair 41: OK
Processing pair 42: OK
Processing p

### Calculating distortions

In [10]:
%%time
print("Calibrating left camera...")
_, left_matrix, left_distortion_coeff, left_rotation_vec, left_translation_vec = cv.calibrateCamera(
    left_obj_points, 
    left_image_points,
    SIZE, None, None
)
print("Matrix:")
print("{:10.3f}\t{:10.3f}\t{:10.3f}".format(left_matrix[0,0], left_matrix[0,1], left_matrix[0,2]))
print("{:10.3f}\t{:10.3f}\t{:10.3f}".format(left_matrix[1,0], left_matrix[1,1], left_matrix[1,2]))
print("{:10.3f}\t{:10.3f}\t{:10.3f}".format(left_matrix[2,0], left_matrix[2,1], left_matrix[2,2]))
print("Distortion coefficients:")
print(left_distortion_coeff)

print("Calibrating right camera...")
_, right_matrix, right_distortion_coeff, right_rotation_vec, right_translation_vec = cv.calibrateCamera(
    right_obj_points, 
    right_image_points,
    SIZE, None, None
)
print("Matrix:")
print("{:10.3f}\t{:10.3f}\t{:10.3f}".format(right_matrix[0,0], right_matrix[0,1], right_matrix[0,2]))
print("{:10.3f}\t{:10.3f}\t{:10.3f}".format(right_matrix[1,0], right_matrix[1,1], right_matrix[1,2]))
print("{:10.3f}\t{:10.3f}\t{:10.3f}".format(right_matrix[2,0], right_matrix[2,1], right_matrix[2,2]))
print("Distortion coefficients:")
print(right_distortion_coeff)

print("OK")

Calibrating left camera...
Matrix:
  1102.948	     0.000	   647.721
     0.000	  1116.223	   536.900
     0.000	     0.000	     1.000
Distortion coefficients:
[[-0.05566086  0.20138652  0.00403251 -0.00098626 -0.27562775]]
Calibrating right camera...
Matrix:
  1102.891	     0.000	   637.789
     0.000	  1115.132	   549.358
     0.000	     0.000	     1.000
Distortion coefficients:
[[-0.09434038  0.3271362   0.00313088 -0.00122388 -0.42300538]]
OK
Wall time: 7min 12s


### Image undistortion

In [12]:
new_left_matrix, left_roi = cv.getOptimalNewCameraMatrix(left_matrix, left_distortion_coeff, SIZE, 0)
new_right_matrix, right_roi = cv.getOptimalNewCameraMatrix(right_matrix, right_distortion_coeff, SIZE, 0)

for f in os.listdir("calibration/left"):
    image = cv.imread("calibration/left/" + f)
    image = cv.undistort(image, left_matrix, left_distortion_coeff, None, new_left_matrix)
    cv.imwrite("calibrate-undistort/left/" + f, image)
    x, y, w, h = left_roi
    cv.imwrite("calibrate-undistort-crop/left/" + f, image[y:y + h, x:x + w])
    
for f in os.listdir("calibration/right"):
    image = cv.imread("calibration/right/" + f)
    image = cv.undistort(image, right_matrix, right_distortion_coeff, None, new_right_matrix)
    cv.imwrite("calibrate-undistort/right/" + f, image)
    x, y, w, h = right_roi
    cv.imwrite("calibrate-undistort-crop/right/" + f, image[y:y + h, x:x + w])

In [26]:
left_error = 0
for i in range(len(left_obj_points)):
    left_image_points2, _ = cv.projectPoints(left_obj_points[i], left_rotation_vec[i], left_translation_vec[i], left_matrix, left_distortion_coeff)
    error = cv.norm(left_image_points[i], left_image_points2, cv.NORM_L2) / len(left_image_points2)
    print("Left", i, ":\t", error)
    left_error += error

right_error = 0
for i in range(len(right_obj_points)):
    right_image_points2, _ = cv.projectPoints(right_obj_points[i], right_rotation_vec[i], right_translation_vec[i], right_matrix, right_distortion_coeff)
    error = cv.norm(right_image_points[i], right_image_points2, cv.NORM_L2) / len(right_image_points2)
    print("Right", i, ":\t", error)
    right_error += error


print("Left re-projection error:", left_error / len(left_obj_points))
print("Right re-projection error:", right_error / len(right_obj_points))

Left 0 :	 0.20556398475011378
Left 1 :	 0.06830732720102646
Left 2 :	 0.0815890390939854
Left 3 :	 0.09926493353149417
Left 4 :	 0.09244734614174467
Left 5 :	 0.08529829354775387
Left 6 :	 0.08352161421371185
Left 7 :	 0.07881371565215714
Left 8 :	 0.08290951163442285
Left 9 :	 0.08364825841140942
Left 10 :	 0.08991964181342028
Left 11 :	 0.05612866533418383
Left 12 :	 0.08977939196723826
Left 13 :	 0.06534923048455828
Left 14 :	 0.06470197011468623
Left 15 :	 0.060976211072288206
Left 16 :	 0.05249702302928571
Left 17 :	 0.05720369600868841
Left 18 :	 0.07061316630830636
Left 19 :	 0.05492727681153911
Left 20 :	 0.06070203246044764
Left 21 :	 0.06688782661577397
Left 22 :	 0.06219007302822925
Left 23 :	 0.08632589937367764
Left 24 :	 0.08609326200160883
Left 25 :	 0.08490831271725412
Left 26 :	 0.07654901242306944
Left 27 :	 0.07807510788913363
Left 28 :	 0.06822144805250317
Left 29 :	 0.08503222831298823
Left 30 :	 0.08053006849465189
Left 31 :	 0.07685663817476485
Left 32 :	 0.08322

In [28]:
%%time
print("Calibrating cameras together...")
_, _, _, _, _, rotation_matrix, translation_vector, _, _ = cv.stereoCalibrate(
    obj_pair_points,
    left_image_pair_points, right_image_pair_points,
    left_matrix, left_distortion_coeff,
    right_matrix, right_distortion_coeff,
    SIZE, None, None, None, None,
    cv.CALIB_FIX_INTRINSIC, CRITERIA
)

Calibrating cameras together...
Wall time: 29.7 s


In [29]:
%%time
print("Rectifying cameras...")
left_rectification, right_rectification, left_projection, right_projection, _, left_roi, right_roi = cv.stereoRectify(
    left_matrix, left_distortion_coeff,
    right_matrix, right_distortion_coeff,
    SIZE, rotation_matrix, translation_vector,
    None, None, None, None, None,
    cv.CALIB_ZERO_DISPARITY, 0.25,
)

Rectifying cameras...
Wall time: 0 ns


In [30]:
left_max_x, left_map_y = cv.initUndistortRectifyMap(
    left_matrix, left_distortion_coeff, left_rectification,
    left_projection, SIZE, cv.CV_32FC1
)

right_max_x, right_map_y = cv.initUndistortRectifyMap(
    right_matrix, right_distortion_coeff, right_rectification,
    right_projection, SIZE, cv.CV_32FC1
)

np.savez_compressed("stereo-12cm.npz", 
    left_max_x=left_max_x, left_map_y=left_map_y, left_roi=left_roi, left_matrix=left_matrix, left_distortion_coeff=left_distortion_coeff, 
    right_max_x=right_max_x, right_map_y=right_map_y, right_roi=right_roi, right_matrix=right_matrix, right_distortion_coeff=right_distortion_coeff,
)

In [4]:
calibration = np.load("stereo-12cm.npz", allow_pickle=False)

left_max_x = calibration["left_max_x"]
left_map_y = calibration["left_map_y"]
left_roi = tuple(calibration["left_roi"])
left_matrix = calibration["left_matrix"]
left_distortion_coeff = calibration["left_distortion_coeff"]

right_max_x = calibration["right_max_x"]
right_map_y = calibration["right_map_y"]
right_roi = tuple(calibration["right_roi"])
right_matrix = calibration["right_matrix"]
right_distortion_coeff = calibration["right_distortion_coeff"]

In [46]:
stereo = cv.StereoBM_create()
stereo.setMinDisparity(4)
stereo.setNumDisparities(128)
stereo.setBlockSize(21)
stereo.setSpeckleRange(16)
stereo.setSpeckleWindowSize(45)

def get_depth(left, right):
    fixed_left = cv.remap(left, left_max_x, left_map_y, cv.INTER_LINEAR)
    fixed_right = cv.remap(right, right_max_x, right_map_y, cv.INTER_LINEAR)

    fixed_left = cv.cvtColor(fixed_left, cv.COLOR_BGR2GRAY)
    fixed_right = cv.cvtColor(fixed_right, cv.COLOR_BGR2GRAY)

    depth = stereo.compute(fixed_left, fixed_right)

    return depth
    # lower, upper = 0, 255
    # return (lower + (upper - lower) * depth).astype(np.uint8)

In [53]:
import IPython

def depth_stream():
    camera0 = cv.VideoCapture(get_source(0))
    camera0.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
    camera0.set(cv.CAP_PROP_FRAME_HEIGHT, 1024)

    camera1 = cv.VideoCapture(get_source(1))
    camera1.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
    camera1.set(cv.CAP_PROP_FRAME_HEIGHT, 1024)
    try: 
        while True:
            camera0.grab()
            camera1.grab()

            _, frame0 = camera0.retrieve()
            _, frame1 = camera1.retrieve()
            
            frame0 = cv.GaussianBlur(frame0, (5, 5), 1000)
            frame1 = cv.GaussianBlur(frame1, (5, 5), 1000)

            kernel = np.ones((20, 20), dtype=np.uint8)
            #frame0 = cv.erode(frame0, kernel, iterations=1)
            #frame1 = cv.erode(frame1, kernel, iterations=1)

            depth = get_depth(frame0, frame1)
            
            frame0 = cv.cvtColor(frame0, cv.COLOR_BGR2GRAY)
            frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

            frame = np.hstack((frame1, frame0))

            # frame = cv.flip(frame, 1)

            kernel = np.ones((5, 5), dtype=np.uint8)
            depth = cv.dilate(depth, kernel)
            DEPTH_VISUALIZATION_SCALE = 2048
            cv.imshow('depth', depth / DEPTH_VISUALIZATION_SCALE)
            cv.waitKey(1)

            # stream = BytesIO()
            # Image.fromarray(frame).save(stream, format="jpeg")
            # IPython.display.display(IPython.display.Image(data=stream.getvalue()))
            # IPython.display.clear_output(wait=True)
    except KeyboardInterrupt:
        camera0.release()
        camera1.release()
        print("Stream Stopped")

In [54]:
depth_stream()

Stream Stopped
